In [11]:
import pymcx as mcx
import json
import random
import SimpleITK as sitk
import numpy as np
import jdata

import pygalmesh
import meshio

mcxbin="../../MCXStudiov2020/MCXSuite/mcx/bin/mcx"

mmcbin="../../MCXStudiov2020/MCXSuite/mmc/bin/mmc"

In [12]:
def better_write_vol_bin(vol, filename, used_type=np.byte):
    f = open(filename, 'wb')
    dmedium = np.array(vol,used_type)
    f.write(dmedium)
    f.close()

In [13]:
img = sitk.ReadImage("../down128.mhd")

dimension = list(img.GetSize())
x = dimension[0]
y = dimension[1]
z = dimension[2]

vol = sitk.GetArrayFromImage(img)

vol = np.where(vol == 0, 1, vol)

vol[:,:,0] = 0
vol[:,0,:] = 0
vol[0,:,:] = 0
vol[:,:,x-1] = 0
vol[:,y-1,:] = 0
vol[z-1,:,:] = 0

vol = vol.astype(np.byte)

binfile = "down128.bin"
better_write_vol_bin(vol, binfile)

In [14]:
vol.shape[2]

128

In [15]:

cfg = mcx.create() #create a default config dictionary

cfg["Domain"]["VolumeFile"] = binfile
cfg["Domain"]["MediaFormat"] = "byte"
cfg["Domain"]["Dim"] = dimension
cfg["Domain"]["OriginType"] = 1
cfg["Domain"]["LengthUnit"] = (1024 / x) * 0.02
cfg["Domain"]["Media"] = [{'mua': 0.0, 'mus': 0.0, 'g': 1.0, 'n': 1.0},
                          {'mua': 0.0, 'mus': 0.0, 'g': 1.0, 'n': 1.0},
                         {'mua': 0.1, 'mus':2.867 , 'g': 0.99, 'n': 1.63},
                         {'mua': 0.35, 'mus': 22.193, 'g': 0.83, 'n': 1.49},
                         {'mua':  2.8, 'mus': 0.0, 'g': 1.0, 'n': 1.333}]


cfg["Forward"]["Dt"] = 5e-09
cfg["Forward"]["T0"] = 0.0
cfg["Forward"]["T1"] = 5e-09


cfg["Session"]["Photons"] = 1e7
cfg["Session"]["RNGSeed"] = random.randint(0,999999999)
cfg["Session"]["ID"] = "cfg_from_python"
cfg["Session"]["DoAutoThread"] = 1


cfg["Optode"]["Source"]["Dir"] =  [0.0, 1, 0]
cfg["Optode"]["Source"]["Param1"] = [0.0, 0.0, 0.0, 0.0]
cfg["Optode"]["Source"]["Param2"] = [0.0, 0.0, 0.0, 0.0]
cfg["Optode"]["Source"]["Pos"] = [x/2, 1,z*3/4]
cfg["Optode"]["Source"]["Type"] = "pencil"

cfg["Optode"]["Detector"] = [{"Pos": [x/2,  y/2,  0.0], "R": 5.0}]


#del cfg["Optode"]["Detector"]
del cfg["Shapes"]


flags = ""
#flags += "-a 1 "
#flags += "--outputformat jnii " # tx3 jnii
flags += "--saveexit 1 "
flags += "--saveref 1 "
flags += "--savedetflag 127 "
#flags += "--bc aaaaaa111111 "
#flags += "--dumpjson"

data = mcx.run(cfg, flag=flags, mcxbin=mcxbin)

###############################################################################
#                      Monte Carlo eXtreme (MCX) -- CUDA                      #
#          Copyright (c) 2009-2020 Qianqian Fang <q.fang at neu.edu>          #
#                             http://mcx.space/                               #
#                                                                             #
# Computational Optics & Translational Imaging (COTI) Lab- http://fanglab.org #
#   Department of Bioengineering, Northeastern University, Boston, MA, USA    #
###############################################################################
#    The MCX Project is funded by the NIH/NIGMS under grant R01-GM114365      #
###############################################################################
$Rev::0313d4$ v2020 $Date::2020-09-06 23:25:03 -04$ by $Author::Qianqian Fang $
###############################################################################
- variant name: [Fermi] compiled by nvcc

In [16]:
jnii = jdata.load("cfg_from_python.jnii")


In [17]:
jdat = jdata.load("cfg_from_python_detp.jdat")

In [18]:
print(jdat["MCXData"]["PhotonData"]["detid"].shape)
print(jdat["MCXData"]["PhotonData"]["nscat"].shape)
print(jdat["MCXData"]["PhotonData"]["ppath"].shape)
print(jdat["MCXData"]["PhotonData"]["mom"].shape)
print(jdat["MCXData"]["PhotonData"]["p"].shape)
print(jdat["MCXData"]["PhotonData"]["v"].shape)
print(jdat["MCXData"]["PhotonData"]["w0"].shape)

#jdat["MCXData"]["PhotonData"]

(28365, 1)
(28365, 4)
(28365, 4)
(28365, 4)
(28365, 3)
(28365, 3)
(28365, 1)


In [19]:
all_layers = jnii["NIFTIData"]

img = sitk.GetImageFromArray(all_layers)
sitk.WriteImage(img, "blub.mhd")

In [24]:
fluence_and_dref = data[1]
dref = np.where(fluence_and_dref < 0, -fluence_and_dref, 0)
dref = np.where(dref == 0, 10**-20, dref)
dref = np.log(dref)
dref = np.where(dref < 0,0, dref)

dref_img = sitk.GetImageFromArray(dref)
sitk.WriteImage(dref_img, "dref.mhd")

In [23]:
dref

array([[[[-46.05170186],
         [-46.05170186],
         [-46.05170186],
         ...,
         [-46.05170186],
         [-46.05170186],
         [-46.05170186]],

        [[-46.05170186],
         [ 10.0113665 ],
         [ 10.03352214],
         ...,
         [ 10.96587272],
         [ 10.80722015],
         [-46.05170186]],

        [[-46.05170186],
         [ 10.2503209 ],
         [  9.84739669],
         ...,
         [ 10.95554366],
         [ 10.89416179],
         [-46.05170186]],

        ...,

        [[-46.05170186],
         [  6.98266253],
         [  6.91745527],
         ...,
         [  6.87620124],
         [  6.06879293],
         [-46.05170186]],

        [[-46.05170186],
         [  7.07834837],
         [  7.39825542],
         ...,
         [  6.19049143],
         [  6.68813016],
         [-46.05170186]],

        [[-46.05170186],
         [-46.05170186],
         [-46.05170186],
         ...,
         [-46.05170186],
         [-46.05170186],
         [-46.051

In [9]:
vol_no_pad = np.where(vol == 0, 1, vol)
vol_no_pad = vol_no_pad.astype(np.uint8)
voxel_size = 1#(1024 / x) * 0.02
voxel_sizes = (voxel_size, voxel_size, voxel_size)

mesh = pygalmesh.generate_from_array(vol_no_pad, 
                                     voxel_sizes,
                                     max_facet_distance=voxel_size, 
                                     max_cell_circumradius=1.0)
mesh.write("tooth.vtk")